<a href="https://colab.research.google.com/github/Chinmaya-3141/Capstone-Project/blob/main/Capstone_Classification_Pretrained_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import time
import numpy as np
from numba import cuda

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from torchvision import datasets, transforms

In [ ]:
# Check if GPU is available

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
file = '/content/drive/MyDrive/Capstone Data (Shared)/Capstone Data/TrainedModelCheckpoints/AdamW-lr001-Batch64-15epochs.pth'

In [ ]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torchinfo import summary

In [ ]:
checkpoint = torch.load(file)

In [ ]:
# Define the CNN model

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv_layer = nn.Sequential(
            # input layer
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            
            # Hidden Layer
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),

            # Hidden Layer
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            
            # Hidden Layer
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )
        self.fc_layer = nn.Sequential(
            # Hidden Layer
            nn.Linear(128 * 20 * 20, 512),
            nn.ReLU(inplace=True),

            # Hidden Layer
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),

            # Output Layer
            nn.Linear(256, 3)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x

In [ ]:
model2 = CNNModel().to(device)
optimizer2 = optim.Adam(model2.parameters(),lr=0.001)

In [ ]:
model2.load_state_dict(checkpoint['model_state_dict'])
optimizer2.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint["epoch"]
loss = checkpoint['loss']

In [ ]:
summary(model2,input_size = (64,3,320,320))

Layer (type:depth-idx)                   Output Shape              Param #
CNNModel                                 [64, 3]                   --
├─Sequential: 1-1                        [64, 128, 20, 20]         --
│    └─Conv2d: 2-1                       [64, 16, 320, 320]        448
│    └─ReLU: 2-2                         [64, 16, 320, 320]        --
│    └─MaxPool2d: 2-3                    [64, 16, 160, 160]        --
│    └─Conv2d: 2-4                       [64, 32, 160, 160]        4,640
│    └─ReLU: 2-5                         [64, 32, 160, 160]        --
│    └─MaxPool2d: 2-6                    [64, 32, 80, 80]          --
│    └─Conv2d: 2-7                       [64, 64, 80, 80]          18,496
│    └─ReLU: 2-8                         [64, 64, 80, 80]          --
│    └─MaxPool2d: 2-9                    [64, 64, 40, 40]          --
│    └─Conv2d: 2-10                      [64, 128, 40, 40]         73,856
│    └─ReLU: 2-11                        [64, 128, 40, 40]         --
│  

In [ ]:
# Set path here, access file path changes person to person

path = '/content/drive/MyDrive/Capstone Data (Shared)/Capstone Data/Resized Data/Resized Data_320x320'

In [ ]:
# Define the transform for the data

transform = transforms.Compose([
    transforms.Resize((320, 320)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
# Load the data from the combined folder

data = ImageFolder(root=path, transform=transform)

In [ ]:
data.classes

['Covid', 'Normal', 'Pneumonia']

In [ ]:
data.class_to_idx

{'Covid': 0, 'Normal': 1, 'Pneumonia': 2}

In [ ]:
# Split the data into training and testing sets

train_data, test_data = torch.utils.data.random_split(data, [int(0.8*len(data)), len(data)-int(0.8*len(data))])

In [ ]:
# Define the data loaders

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [ ]:
# Evaluate the model on test data

correct = 0
total = 0
predictions=[]
actuals=[]
with torch.no_grad():
    for datavar in test_loader:
        images, labels = datavar
        images,labels = images.to(device),labels.to(device)
        outputs = model2(images)
        predicted = torch.argmax(outputs.datavar, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        pred_arr = predicted.cpu().detach().numpy()
        actual_arr = labels.cpu().detach().numpy()
        predictions.append(pred_arr)
        actuals.append(actual_arr)

print('Accuracy of the network on the test images: %f %%'% (100 * correct / total))

Accuracy of the network on the test images: 99.347684 %


In [ ]:
# predictions

[array([1, 1, 1, 0, 1, 1, 1, 2, 0, 0, 2, 2, 0, 0, 1, 1, 0, 1, 2, 1, 1, 0,
        0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 1, 1,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 1]),
 array([0, 1, 1, 1, 1, 0, 0, 0, 2, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 2, 0, 0,
        1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        2, 1, 2, 2, 1, 0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0]),
 array([0, 2, 0, 1, 0, 2, 2, 2, 2, 1, 0, 1, 2, 0, 1, 0, 2, 0, 2, 0, 2, 2,
        0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 1, 1, 1, 0, 2, 0, 1, 0, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 2]),
 array([1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 0, 1, 1, 0, 1, 0, 2, 0, 1, 0, 0, 1,
        1, 2, 0, 2, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 2, 1, 0, 2, 2,
        0, 0, 1, 1, 0, 1, 2, 1, 1, 2, 2, 0, 0, 2, 0, 0, 2, 1, 1, 0]),
 array([0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 1, 1, 0, 2, 0, 1, 2, 2, 1, 0, 2,
        0, 0, 1, 0, 2, 0, 0, 1, 0, 0, 2, 1, 1, 0, 0, 1

In [ ]:
pred=np.concatenate(predictions)

In [ ]:
print(pred)

[1 1 1 ... 1 1 0]


In [ ]:
# actuals

[array([1, 1, 1, 0, 1, 1, 1, 2, 0, 0, 2, 2, 0, 0, 1, 1, 0, 1, 2, 1, 1, 0,
        0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 1, 1,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2, 1, 1, 0, 0, 0, 0, 0, 2, 1]),
 array([0, 1, 1, 1, 1, 0, 0, 0, 2, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 2, 0, 0,
        1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 2, 1, 0, 1, 1, 1, 1, 1, 2, 0, 1, 1,
        2, 1, 2, 2, 1, 0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0]),
 array([0, 2, 0, 1, 0, 2, 2, 2, 2, 1, 0, 1, 2, 0, 1, 0, 2, 0, 2, 0, 2, 2,
        0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 1, 1, 1, 0, 2, 0, 1, 0, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 2, 2]),
 array([1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 0, 1, 1, 0, 1, 0, 2, 0, 1, 0, 0, 1,
        1, 2, 0, 2, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 2, 1, 0, 2, 2,
        0, 0, 1, 1, 0, 1, 2, 1, 1, 2, 2, 0, 0, 2, 0, 0, 2, 1, 1, 0]),
 array([0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 1, 1, 0, 2, 0, 1, 2, 2, 1, 0, 2,
        0, 0, 1, 0, 2, 0, 0, 1, 0, 0, 2, 1, 1, 0, 0, 1

In [ ]:
act=np.concatenate(actuals)

In [ ]:
print(act)

[1 1 1 ... 1 1 0]


In [ ]:
pred_arr = predicted.cpu().detach().numpy()

In [ ]:
actual_arr = labels.cpu().detach().numpy()

In [ ]:
import pandas as pd

In [ ]:
pd.crosstab(pred,act,rownames=['Predicted'], colnames=['Actual'])

Actual,0,1,2
Predicted,,,
0,666,2,1
1,2,602,3
2,1,1,255


In [ ]:
data.classes

AttributeError: ignored

In [ ]:
data.class_to_idx